In [92]:
import pandas as pd
churn = pd.read_csv("/Users/sidpai/Desktop/Sem_1/Data_Mining_1/Project/dataset/train.csv")
churn.head()

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod,Churn
0,55,19.50,1026.35,Male,0,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,0
1,72,25.85,1872.2,Male,0,Yes,No,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),0
2,1,75.90,75.9,Male,0,No,No,Yes,No,Fiber optic,No,No,No,Yes,No,No,Month-to-month,Yes,Electronic check,1
3,32,79.30,2570,Female,1,Yes,No,Yes,Yes,Fiber optic,No,No,Yes,No,No,No,Month-to-month,No,Mailed check,0
4,60,115.25,6758.45,Female,0,Yes,Yes,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Credit card (automatic),0


In [93]:
# PRE-PROFESSING TRAINING DATASET

# pre_processing fields like Sex, HasPartner, HasChild etc
from sklearn import preprocessing

# make copty of original training set
churn_preprocessed = churn.copy()

# set output column to a separate dataframe to load into classifier
churn_target = churn['Churn']

# no need to encode target variable to numeric because it is already done
# encode HasPartner, HasChild, HasPhoneService, HasMultiplePhoneNumbers, IsBillingPaperless to (0,1) for (No, Yes)
label_encoder = preprocessing.LabelEncoder()
churn_Sex = pd.DataFrame(label_encoder.fit_transform(churn['Sex']), columns=['Sex_encoded'])
churn_HasPartner = pd.DataFrame(label_encoder.fit_transform(churn['HasPartner']), columns=['HasPartner_encoded'])
churn_HasChild = pd.DataFrame(label_encoder.fit_transform(churn['HasChild']), columns=['HasChild_encoded'])
churn_HasPhoneService = pd.DataFrame(label_encoder.fit_transform(churn['HasPhoneService']), columns=['HasPhoneService_encoded'])
churn_IsBillingPaperless = pd.DataFrame(label_encoder.fit_transform(churn['IsBillingPaperless']), columns=['IsBillingPaperless_encoded'])

# encode HasInternetService, HasOnlineSecurityService, HasOnlineBackup, HasDeviceProtection, HasTechSupportAccess, HasOnlineTV, HasMovieSubscription, HasContractPhone
encoder = preprocessing.OneHotEncoder()
encoded = pd.DataFrame(encoder.fit_transform(churn_preprocessed[['HasMultiplePhoneNumbers','HasInternetService', 'HasOnlineSecurityService','HasOnlineBackup','HasDeviceProtection','HasTechSupportAccess','HasOnlineTV','HasMovieSubscription','HasContractPhone','PaymentMethod']]).toarray(), columns=encoder.get_feature_names_out(['HasMultiplePhoneNumbers','HasInternetService', 'HasOnlineSecurityService','HasOnlineBackup','HasDeviceProtection','HasTechSupportAccess','HasOnlineTV','HasMovieSubscription','HasContractPhone','PaymentMethod']))

# drop the original columns and join the encoded columns
# also drop Churn as it is our target field
churn_preprocessed = churn_preprocessed.drop(columns=['Sex','HasPartner','HasChild','HasPhoneService','IsBillingPaperless','HasMultiplePhoneNumbers','HasInternetService','HasOnlineSecurityService','HasOnlineBackup','HasDeviceProtection','HasTechSupportAccess','HasOnlineTV','HasMovieSubscription','HasContractPhone','PaymentMethod','Churn'])
churn_preprocessed = churn_preprocessed.join(churn_Sex)
churn_preprocessed = churn_preprocessed.join(churn_HasPartner)
churn_preprocessed = churn_preprocessed.join(churn_HasChild)
churn_preprocessed = churn_preprocessed.join(churn_HasPhoneService)
churn_preprocessed = churn_preprocessed.join(churn_IsBillingPaperless)
churn_preprocessed = churn_preprocessed.join(encoded)

# convert TotalSpent from object to float64 and mark NaN as 0.0
churn_preprocessed['TotalSpent'] = pd.to_numeric(churn_preprocessed['TotalSpent'],errors = 'coerce').fillna(0)

In [94]:
from sklearn.naive_bayes import GaussianNB
naive_bayes = GaussianNB()
naive_bayes.fit(churn_preprocessed, churn_target)

GaussianNB()

In [96]:
churn_prediction = naive_bayes.predict(churn_preprocessed)
churn_prediction

array([0, 0, 1, ..., 1, 1, 1])

In [98]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

display(churn_target)
display(churn_prediction)

print("Confusion Matrix:")
print(confusion_matrix(churn_target, churn_prediction))
print()

print("Accuracy: {}".format(accuracy_score(churn_target, churn_prediction)))

0       0
1       0
2       1
3       0
4       0
       ..
5277    0
5278    0
5279    0
5280    0
5281    0
Name: Churn, Length: 5282, dtype: int64

array([0, 0, 1, ..., 1, 1, 1])

Confusion Matrix:
[[2482 1416]
 [ 212 1172]]

Accuracy: 0.6917834153729648


In [84]:
churn_test = pd.read_csv("/Users/sidpai/Desktop/Sem_1/Data_Mining_1/Project/dataset/test.csv")
churn_test.head()

,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod
0,42,56.10,2386.85,Male,0,Yes,No,Yes,Yes,DSL,Yes,No,No,No,No,No,One year,No,Credit card (automatic)
1,29,98.50,3004.15,Male,1,Yes,No,Yes,Yes,Fiber optic,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check
2,9,76.25,684.85,Male,0,No,No,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check
3,67,74.00,4868.4,Female,0,Yes,Yes,Yes,Yes,DSL,Yes,Yes,No,Yes,Yes,No,Two year,No,Credit card (automatic)
4,71,20.10,1389.6,Female,0,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Bank transfer (automatic)


In [86]:
# PRE-PROFESSING TESTING DATASET

# pre_processing fields like Sex, HasPartner, HasChild etc

# make copty of original training set
churn_preprocessed_test = churn_test.copy()

# no need to encode target variable to numeric because it is already done
# encode HasPartner, HasChild, HasPhoneService, HasMultiplePhoneNumbers, IsBillingPaperless to (0,1) for (No, Yes)
label_encoder = preprocessing.LabelEncoder()
churn_Sex = pd.DataFrame(label_encoder.fit_transform(churn_test['Sex']), columns=['Sex_encoded'])
churn_HasPartner = pd.DataFrame(label_encoder.fit_transform(churn_test['HasPartner']), columns=['HasPartner_encoded'])
churn_HasChild = pd.DataFrame(label_encoder.fit_transform(churn_test['HasChild']), columns=['HasChild_encoded'])
churn_HasPhoneService = pd.DataFrame(label_encoder.fit_transform(churn_test['HasPhoneService']), columns=['HasPhoneService_encoded'])
churn_IsBillingPaperless = pd.DataFrame(label_encoder.fit_transform(churn_test['IsBillingPaperless']), columns=['IsBillingPaperless_encoded'])

# encode HasInternetService, HasOnlineSecurityService, HasOnlineBackup, HasDeviceProtection, HasTechSupportAccess, HasOnlineTV, HasMovieSubscription, HasContractPhone
encoder = preprocessing.OneHotEncoder()
encoded = pd.DataFrame(encoder.fit_transform(churn_preprocessed_test[['HasMultiplePhoneNumbers','HasInternetService', 'HasOnlineSecurityService','HasOnlineBackup','HasDeviceProtection','HasTechSupportAccess','HasOnlineTV','HasMovieSubscription','HasContractPhone','PaymentMethod']]).toarray(), columns=encoder.get_feature_names_out(['HasMultiplePhoneNumbers','HasInternetService', 'HasOnlineSecurityService','HasOnlineBackup','HasDeviceProtection','HasTechSupportAccess','HasOnlineTV','HasMovieSubscription','HasContractPhone','PaymentMethod']))

# drop the original columns and join the encoded columns
churn_preprocessed_test = churn_preprocessed_test.drop(columns=['Sex','HasPartner','HasChild','HasPhoneService','IsBillingPaperless','HasMultiplePhoneNumbers','HasInternetService','HasOnlineSecurityService','HasOnlineBackup','HasDeviceProtection','HasTechSupportAccess','HasOnlineTV','HasMovieSubscription','HasContractPhone','PaymentMethod'])
churn_preprocessed_test = churn_preprocessed_test.join(churn_Sex)
churn_preprocessed_test = churn_preprocessed_test.join(churn_HasPartner)
churn_preprocessed_test = churn_preprocessed_test.join(churn_HasChild)
churn_preprocessed_test = churn_preprocessed_test.join(churn_HasPhoneService)
churn_preprocessed_test = churn_preprocessed_test.join(churn_IsBillingPaperless)
churn_preprocessed_test = churn_preprocessed_test.join(encoded)

# convert TotalSpent from object to float64 and mark NaN as 0.0
churn_preprocessed_test['TotalSpent'] = pd.to_numeric(churn_preprocessed_test['TotalSpent'],errors = 'coerce').fillna(0)

In [91]:
churn_prediction = naive_bayes.predict(churn_preprocessed_test)
churn_prediction

[0 1 1 ... 1 1 0]
